<a href="https://colab.research.google.com/github/xiu0327/lab/blob/main/cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

In [ ]:
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

In [ ]:
print(x_train[0])
print(y_train[0])

In [ ]:
pip install konlpy

In [ ]:
#영화 리뷰 데이터 전처리

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data = pd.read_table('/content/ratings_train.txt')
test_data = pd.read_table('/content/ratings_test.txt')

In [ ]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

In [ ]:
train_data[:5]

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

In [ ]:
print('총 샘플의 수 :',len(train_data))

In [ ]:
print(train_data.groupby('label').size().reset_index(name = 'count'))
# 0 : 부정 리뷰 개수 / 1 : 긍정 리뷰 개수

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "")
#공백 -> empyty value로 변경
train_data['document'].replace('', np.nan, inplace=True)

In [ ]:
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
train_data = train_data.dropna(how = 'any')
#null 데이터 제거

In [ ]:
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

In [ ]:
#불용어 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()

In [ ]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
word_cnt = 3
total_cnt = len(tokenizer.word_index) # 정수화된 단어의 총 개수
rare_cnt = 0 # 빈도수 word_cnt 이하 단어의 개수
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 word_cnt보다 작은 단어의 등장 빈도수 총 합

for key, value in tokenizer.word_counts.items():
    total_freq += value
    if(value < word_cnt):
        rare_cnt +=1
        rare_freq += value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(word_cnt - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
# 텍스트 -> 숫자 시퀀스로 변환
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[19415])

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
# 문장 패딩
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))

In [ ]:
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

In [ ]:
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)
# 모든 리뷰의 길이를 30으로 맞춤

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# cnn-lstm 모델 설정
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)


In [ ]:
pip install load_model

In [ ]:
from tensorflow.keras.models import load_model
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  print(pad_new)
  score = float(model.predict(pad_new)) # 예측
  if(score > 0.5):
    #print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    return "{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100)
  else:
    #print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
    return "{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100)

In [ ]:
sentiment_predict("이 영화 개꿀잼 ㅋㅋㅋ")

In [ ]:
# 리뷰 예측

outputData = open('/content/outputData.txt', 'w', encoding='utf-8')

y_pridict = []
pos = []
neg = []

sentence = pd.read_table('/content/ratings_test.txt')
data = sentence['document']

In [ ]:
print(data[1])

In [ ]:
    for sen in data:
        print(sen)
        try:
            new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', sen)
            new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
            new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
            encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
            pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
            score = float(model.predict(pad_new)) # 예측
            if score > 0.5:
                outputData.write("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
                y_pridict.append(1)
                pos.append(score)
            else:
                outputData.write("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
                y_pridict.append(0)
                neg.append(score)
        except:
            y_pridict.append(0)
            continue

In [ ]:
print(len(y_pridict))
label = sentence['label']
print(len(label))